# Playbooks

Describe in more details playbooks.

Show some example playbooks with:

 - user and skel
 - install packages
 - enable service #not on containers
 - lineinfile
 - curl
 - file
 - copy
 - fetch
 
Creating small reports.

More fun with:

  - iterations
  - ignore_errors
  - when
  - with_fileglob and remote_fileglob ;)

In [41]:
cd exercise-06

[Errno 2] No such file or directory: 'exercise-06'
/notebooks/exercise-06


## Gathering facts

playbooks are groups of tasks we can run with ansible.

A playbook starts with an hosts mark specifying the hosts to run the playbook to.

```
---
- hosts: web
  tasks: 
  ...
```

When run, a playbook gathers facts about hosts (remember the setup module?).

Gathering facts may be time-consuming, so you can disabe that with

```
- hosts: web
  gather_facts: no
  tasks:
  ...
```



In [ ]:
# %load debug.yml
- hosts: localhost
  name: those tasks are run on localhost only
  tasks:
  - name: This is the debug module.
    debug: msg="A string"
    
  - name: The debug module can print a variable too
    debug: var=ansible_hostname

  - name: We can iterate too
    debug: msg="{{item}}"
    with_items: "{{ansible_system_capabilities}}"

  - name: and inspect/format variables
    debug: msg="The device is {{item.device}}"
    with_items: >
      {{ansible_mounts}}



In [23]:
!ansible-playbook debug.yml


PLAY [those tasks are run on localhost only] ***********************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [This is the debug module.] ***********************************************
ok: [localhost] => {
    "msg": "A string"
}

TASK [The debug module can print a variable too] *******************************
ok: [localhost] => {
    "ansible_hostname": "sysadminpy"
}

TASK [We can iterate too] ******************************************************
ok: [localhost] => (item=cap_chown) => {
    "item": "cap_chown", 
    "msg": "cap_chown"
}
ok: [localhost] => (item=cap_dac_override) => {
    "item": "cap_dac_override", 
    "msg": "cap_dac_override"
}
ok: [localhost] => (item=cap_fowner) => {
    "item": "cap_fowner", 
    "msg": "cap_fowner"
}
ok: [localhost] => (item=cap_fsetid) => {
    "item": "cap_fsetid", 
    "msg": "cap_fsetid"
}
ok: [localhost] => (item=cap_kill) => {
    "item": "cap_kill", 
    "msg"

## Exercise: 

[modify debug.yml](/edit/notebooks/exercise-06/debug.yml) to disable fact gathering and use the following cell to test it 

In [24]:
!ansible-playbook debug.yml


PLAY [those tasks are run on localhost only] ***********************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [This is the debug module.] ***********************************************
ok: [localhost] => {
    "msg": "A string"
}

TASK [The debug module can print a variable too] *******************************
ok: [localhost] => {
    "ansible_hostname": "sysadminpy"
}

TASK [We can iterate too] ******************************************************
ok: [localhost] => (item=cap_chown) => {
    "item": "cap_chown", 
    "msg": "cap_chown"
}
ok: [localhost] => (item=cap_dac_override) => {
    "item": "cap_dac_override", 
    "msg": "cap_dac_override"
}
ok: [localhost] => (item=cap_fowner) => {
    "item": "cap_fowner", 
    "msg": "cap_fowner"
}
ok: [localhost] => (item=cap_fsetid) => {
    "item": "cap_fsetid", 
    "msg": "cap_fsetid"
}
ok: [localhost] => (item=cap_kill) => {
    "item": "cap_kill", 
    "msg"

### Hints

 - use ansible_facts as possible instead of gathering facts with uname & co
 - test and template your iterations statically instead of continuosly gathering facts
 

In [25]:
%load process-facts.yml



ansible.cfg  debug.yml	inventory  process-facts.retry	process-facts.yml


In [27]:
!ansible-playbook process-facts.yml | grep msg

    "msg": "/dev/mapper/docker-253:2-537280731-eb92264366fcbaf248d3d5a93bbaab17eec130b43ff94e44e11abab77f5f2038 99%\n"
    "msg": "/dev/mapper/vg0-home 8%\n"


## Packages, User and Files

Creating user and installing packages is easy

```
  - name: Install apache
    apt: item="{{item}}" state=present
    with_items:
    - apache2
    - curl
    
  - name: Remove wget
    apt: item=wget state=absent
```

In [ ]:
# %load package-user.yml
- hosts: web
  tasks:
  - name: Install apache and other packages eventually iterating
    apt: name="{{item}}"
    with_items:
    - apache2
    - curl

  - name: Remove wget
    apt: name=wget state=absent

  - name: Create user
    user: 
      name: foo
      groups: adm
      append: yes
      shell: /bin/bash
      skeleton: /root   # duplicate /root/ content for the foo user

  - name: >
      Copy a file to ~foo expandig tilde.
      Do not forget quotes!
    copy:
      src: package-user.yml
      dest: "{{ '~foo' | expanduser }}"

  - name: >
      We can fetch files too (eg. logs):
      - files are dispatched to different directories.
      - we can avoid enforcing checksum as we're messing with 
        containers.
      
    fetch:
      src: /etc/passwd
      dest: /tmp/fetched
      validate_checksum: no


In [36]:
!ansible-playbook package-user.yml



PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [ansible101_web_1]

TASK [Install apache and other packages eventually iterating] ******************
ok: [ansible101_web_1] => (item=[u'apache2', u'curl'])

TASK [Remove wget] *************************************************************
ok: [ansible101_web_1]

TASK [Create user] *************************************************************
ok: [ansible101_web_1]

TASK [Copy a file to ~foo expandig tilde. Do not forget quotes!] ***************
changed: [ansible101_web_1]

TASK [We can fetch files too (eg. logs): - files are dispatched to different directories. - we can avoid enforcing checksum as we're messing with 
  containers.] ***
changed: [ansible101_web_1]

PLAY RECAP *********************************************************************
ansible101_web_1           : ok=6    changed=2    unreachable=0    failed=0   


In [42]:
!tree /tmp/fetched/ansible101_web_1/etc/

/tmp/fetched/ansible101_web_1/etc/

0 directories, 0 files


## Files and Directories

Creating files and directories. 

lineinfile / blockinfile

with_fileglob

In [48]:
!ansible-playbook -i inventory files-content.yml 



PLAY [A playbook can be made of multiple stanzas. They'll be pipelined.] *******

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [Delete previous template] ************************************************
changed: [localhost]

TASK [We're now just showing how to create a file containing a block of lines] ***
changed: [localhost]

TASK [Register command output in a variable] ***********************************
changed: [localhost]

TASK [Show output] *************************************************************
ok: [localhost] => {
    "o_cat.stdout": "False\n<html> <body> System installed by sysadminpy on {{ ansible_hostname }} </body> </html>\nFalse"
}

PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [ansible101_web_1]

TASK [Install apache and other packages eventually iterating] ******************
ok: [ansible

## Templates

Creating files from templates.

include template files, enforcing policies

tagging

